In [ ]:
import pandas as pd
from constants import *
from sklearn.cluster import KMeans
from flows.specific.custom_flow import  CustomFlow
import os
from data_sources.specific.coca import Coca
from data_sources.specific.gold_stock_price import GoldStockPrice
from data_sources.specific.malaria_cases import Malaria
from embedders.specific.identity_embedder import IdentityEmbbeder
from embedders.specific.aggregation_embedder import AggregationEmbedder
from embedders.specific.linear_regression_coefficient_embedder import LinearRegressionCoefficientEmbedder

from clusterers.specific.identity_clusterer import IdentityClusterer
from clusterers.specific.sklearn_vector_clusterer import SklearnVectorClusterer
from clusterers.specific.quantile_clusterer import QuantileClusterer

from geography.specific.colombian_municipalities import ColombianMunicipalities


In [ ]:
fuentes_incluidas = [Coca]
target_col = 'malaria_num_cases_total'
periodocity = "MONTH"
geo = ColombianMunicipalities()
df_geo = geo.get_geometry()

for data_source in fuentes_incluidas:
    ds = data_source()
    df = ds.createData(df_geo=df_geo, time_resolution=periodocity)

In [ ]:
fuentes_incluidas = [Malaria]
target_col = 'malaria_num_cases_total'
periodocity = "MONTH"
geo = ColombianMunicipalities()
df_geo = geo.get_geometry()

for data_source in fuentes_incluidas:
    ds = data_source()
    df = ds.createData(df_geo=df_geo, time_resolution=periodocity)

In [ ]:
geo

In [ ]:
print(geo.index)
